In [1]:
import numpy as np
import pandas as pd
import sqlite3 as sql3

In [3]:
conn = sql3.connect(':memory:')
cur = conn.cursor()

sql_create_db = open('files/cryptos.sql','r').read()

cur.executescript(sql_create_db)

In [5]:
coins_df = pd.read_sql("SELECT * FROM cryptocoins_cryptocurrency",conn)
coins_df.head()

,id,name,slug,symbol,rank,price_usd,price_btc,volume_usd_24h,market_cap_usd,available_supply,total_supply,max_supply,percent_change_1h,percent_change_24h,percent_change_7d,exchange_id,last_updated
0,1,Bitcoin,bitcoin,BTC,1,8707.37,1.000000,1.874329e+10,157139896329,18046762,18046762,2.100000e+07,-0.12,-1.08,-5.91,1,2019-11-14 13:02:23.293956
1,2,Ethereum,ethereum,ETH,2,186.50,0.021433,7.703154e+09,20244944406,108549236,108549236,NaN,-0.21,-0.33,-0.66,1,2019-11-14 13:02:23.326936
2,3,XRP,ripple,XRP,3,0.27,0.000031,1.320452e+09,11621563439,43298481757,99991298961,1.000000e+11,-0.33,-1.83,-8.22,1,2019-11-14 13:02:23.337368
3,4,Bitcoin Cash,bitcoin-cash,BCH,4,278.92,0.032053,1.759249e+09,5051906817,18112350,18112350,2.100000e+07,0.17,-3.00,-4.76,3,2019-11-14 13:02:23.348683
4,5,Tether,tether,USDT,5,1.01,0.000116,2.172326e+10,4141681919,4108044456,4207771504,NaN,-0.23,0.33,0.45,1,2019-11-14 13:02:23.360212


In [6]:
ex_df = pd.read_sql("SELECT * FROM cryptocoins_exchange",conn)
ex_df.head()

,id,name,url
0,1,Bitstamp,https://www.bitstamp.net
1,2,Poloniex,https://poloniex.com
2,3,Binance,https://www.binance.com/
3,4,OKEx,https://www.okex.com


In [10]:
sql_join = """
    SELECT
        cc.name as coin_name,
        ce.name as exchange,
        cc.symbol,
        cc.price_usd,
        cc.percent_change_7d
    FROM
        cryptocoins_cryptocurrency cc
        JOIN cryptocoins_exchange ce ON ce.id = cc.exchange_id;
"""

crypto_full_df = pd.read_sql(sql_join, conn)
crypto_full_df.head()

,coin_name,exchange,symbol,price_usd,percent_change_7d
0,Bitcoin,Bitstamp,BTC,8707.37,-5.91
1,Ethereum,Bitstamp,ETH,186.50,-0.66
2,XRP,Bitstamp,XRP,0.27,-8.22
3,Bitcoin Cash,Binance,BCH,278.92,-4.76
4,Tether,Bitstamp,USDT,1.01,0.45


In [12]:
weekly_change_df = crypto_full_df.sort_values('percent_change_7d', ascending=False)
weekly_change_df.head(25)

,coin_name,exchange,symbol,price_usd,percent_change_7d
65,DxChain Token,OKEx,DX,0.00,89.14
39,MINDOL,Bitstamp,MIN,1.36,72.90
74,Seele,Bitstamp,SEELE,0.11,39.59
61,Komodo,Poloniex,KMD,0.86,22.75
26,VeChain,OKEx,VET,0.01,21.49
12,Chainlink,Poloniex,LINK,3.15,16.93
16,NEO,OKEx,NEO,12.63,15.92
95,iExec RLC,Bitstamp,RLC,0.73,10.64
94,Aurora,Binance,AOA,0.01,9.49
36,MEXC Token,OKEx,MEXC,0.24,8.74


In [15]:
cur.close()

In [17]:
conn.close()